In [1]:
from torch_geometric.loader import DataLoader

from data import load_mnist_graph

from torch_geometric.nn import global_add_pool, global_mean_pool, global_max_pool

from layers import GraphSAGELayer
from transforms import AddDegreeFeature
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

from train import train_graph_classification, evaluate_graph_classification
from copy import deepcopy
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from torch_geometric.datasets import TUDataset

%reload_ext autoreload
%autoreload 2

In [2]:
# load reddit binary dataset
dataset = TUDataset(root='data',name="REDDIT-BINARY", transform=AddDegreeFeature())

# split dataset into train and test
train_dataset = dataset[:1500]
test_dataset = dataset[1500:]

In [3]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [17]:
class GraphSAGE(nn.Module):
    def __init__(self, input_features, hidden_features, output_features, num_layers):
        super(GraphSAGE, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(GraphSAGELayer(input_features, hidden_features, aggr='sum'))
        for _ in range(num_layers - 1):
            self.layers.append(GraphSAGELayer(hidden_features, hidden_features, aggr='sum'))
        self.linear = nn.Linear(hidden_features, output_features)

        # initialize all weights
        self.reset_parameters()
        
    def forward(self, x, edge_index, batch):
        for layer in self.layers:
            x = layer.propagate(x, edge_index)

        # readout
        x = global_mean_pool(x, batch)

        x = self.linear(x)
        x = F.sigmoid(x)
        return x
    
    def reset_parameters(self):
        for layer in self.layers:
            layer.reset_parameters()
        self.linear.reset_parameters()
    
        
    

In [9]:

### Max number of epochs
max_epochs = 20

### Number of features
n_features, n_classes = 1,2
hidden_size = 64

### DEFINE THE MODEL
basic_model = GraphSAGE(n_features, hidden_size, n_classes, num_layers=2).to(device)

### DEFINE LOSS FUNCTION
loss_fcn = nn.BCEWithLogitsLoss()

### DEFINE OPTIMIZER
optimizer = torch.optim.Adam(basic_model.parameters(), lr=0.001)


### TRAIN THE MODEL
epoch_list, basic_model_scores = train_graph_classification(
    basic_model,
    loss_fcn,
    device,
    optimizer,
    max_epochs,
    train_loader,
    val_loader,
    n_classes,
)

100%|██████████| 20/20 [00:14<00:00,  1.39epoch/s, f1=0.8, loss=0.602]  


In [19]:

# gather all data in one dataset (train/val/test)   
all_data = train_dataset + test_dataset
all_loader = DataLoader(all_data, batch_size=32, shuffle=False)

### Max number of epochs
max_epochs = 40

### Number of features
n_features, n_classes = 1, 1

### DEVICE GPU OR CPU : will select GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("\nDevice: ", device)


K = 12
best_score = 0
best_model = None
best_hyperparameters = None
results = pd.DataFrame(columns=['K', 'valid_score', 'test_score', 'lr', 'num_layers', 'hidden_size'])
for k in range(K):
    print(f"K-Fold: {k+1}/{K}")
    # split the data into train and test
    n = len(all_data)//K 
    test_range_index = np.random.choice(len(all_data), n, replace=False)
    # all the rest for train
    train_range_index = np.setdiff1d(np.arange(len(all_data)), test_range_index)
    train_out = torch.utils.data.Subset(all_data, train_range_index)
    test = torch.utils.data.Subset(all_data, test_range_index)

    # randomly choose 20% of the train_out data to be the validation set
    n = len(train_out)//5
    val_range_index = np.random.choice(len(train_out), n, replace=False)
    val = torch.utils.data.Subset(train_out, val_range_index)
    train_in = torch.utils.data.Subset(train_out, np.setdiff1d(np.arange(len(train_out)), val_range_index))

    

    # create the dataloader
    train_in_loader = DataLoader(train_in, batch_size=64, shuffle=True)
    val_loader = DataLoader(val, batch_size=64, shuffle=False)
    test_loader = DataLoader(test, batch_size=64, shuffle=False)

    # define hyperparameters grid
    hyperparameters = {
        'lr': [0.01, 0.001],
        'num_layers': [2, 3],
        'hidden_size': [16, 32]
    }

    best_score = 0
    for lr in hyperparameters['lr']:
        for num_layers in hyperparameters['num_layers']:
            for hidden_size in hyperparameters['hidden_size']:
                print(f"lr: {lr}, num_layers: {num_layers}, hidden_size: {hidden_size}")
                ### DEFINE THE MODEL
                basic_model = GraphSAGE(n_features, hidden_size, n_classes, num_layers).to(device)

                ### DEFINE LOSS FUNCTION
                loss_fcn = nn.BCELoss()

                ### DEFINE OPTIMIZER
                optimizer = torch.optim.Adam(basic_model.parameters(), lr=lr)
                scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, min_lr=0.0001)

                ### TRAIN THE MODEL
                epoch_list, basic_model_scores = train_graph_classification(
                    basic_model,
                    loss_fcn,
                    device,
                    optimizer,
                    max_epochs,
                    train_in_loader,
                    val_loader,
                    n_classes,
                    save_best=True
                )

                # evaluate the model on the test set
                valid_score = evaluate_graph_classification(basic_model, device, val_loader,n_classes)
                print(f"Valid F1-score: {valid_score}")

                # save the best model
                if valid_score > best_score:
                    best_score = valid_score
                    del best_model
                    best_model = deepcopy(basic_model)
                    best_hyperparameters = {
                        'lr': lr,
                        'num_layers': num_layers,
                        'hidden_size': hidden_size
                    }
                results.loc[len(results.index)] = [k, valid_score, 0, lr, num_layers, hidden_size]

                del basic_model
                del optimizer
                del scheduler
                torch.cuda.empty_cache()
                

    # evaluate the best model on the test set
    test_score = evaluate_graph_classification(best_model, device, test_loader, n_classes)
    print(f"Test F1-score: {test_score}")

    # save the results in dataframe
    results.loc[len(results.index)] = [k, best_score, test_score, best_hyperparameters['lr'], best_hyperparameters['num_layers'], best_hyperparameters['hidden_size']]

# save the results in a csv file
results.to_csv('sage_reddit.csv', index=False)


Device:  cuda
K-Fold: 1/12
lr: 0.01, num_layers: 2, hidden_size: 16


  0%|          | 0/40 [00:00<?, ?epoch/s]

100%|██████████| 40/40 [00:15<00:00,  2.58epoch/s, f1=0.816, loss=0.377]


Best model loaded
Valid F1-score: 0.8528079710144928
lr: 0.01, num_layers: 2, hidden_size: 32


100%|██████████| 40/40 [00:15<00:00,  2.61epoch/s, f1=0.506, loss=50.5]


...

Best model loaded
Valid F1-score: 0.7419610507246377
lr: 0.001, num_layers: 3, hidden_size: 16


100%|██████████| 40/40 [00:15<00:00,  2.55epoch/s, f1=0.629, loss=1.98]


Best model loaded
Valid F1-score: 0.7086730072463768
lr: 0.001, num_layers: 3, hidden_size: 32


100%|██████████| 40/40 [00:17<00:00,  2.35epoch/s, f1=0.51, loss=49.9] 

Best model loaded
Valid F1-score: 0.7606431159420289
Test F1-score: 0.7924890350877193


In [21]:
# see the mean and std of the results for the test set
test_results = results[results['test_score'] != 0].test_score
print(f"Mean test score: {test_results.mean()}")
print(f"Std test score: {test_results.std()}")

Mean test score: 0.8273254751461989
Std test score: 0.036477245025604926
